# Demonstration of qubit subspace techniques using `Symmer`

Here we demonstrate how one arrives at the 3-qubit contextual subspace Hamiltonian that was taken as a testbed for our quantum error mitigation benchmark paper. This utilizes the [symmer](https://github.com/UCL-CCS/symmer) python package, that will need to be installed if one wishes to run these notebooks.

In [54]:
import os
import json
import numpy as np
from matplotlib import pyplot as plt
from symmer.symplectic import PauliwordOp, QuantumState
from symmer.utils import exact_gs_energy

qem_directory =  os.getcwd()
print(qem_directory)
ham_data_dir = os.path.join(qem_directory, 'data/hamiltonian')
qem_data_dir = os.path.join(qem_directory, 'data/QEM_benchmark')

filename = 'HCl_STO-3G_SINGLET_JW.json'
    
with open(os.path.join(ham_data_dir, filename), 'r') as infile:
    data_dict = json.load(infile)

# 20-qubit HCl Hamiltonian:
H = PauliwordOp.from_dictionary(data_dict['hamiltonian'])
# ... the corresponding coupled cluster singles doubles operator:
UCC = PauliwordOp.from_dictionary(data_dict['data']['auxiliary_operators']['UCCSD_operator'])
# ... the number operator:
NUM = PauliwordOp.from_dictionary(data_dict['data']['auxiliary_operators']['number_operator'])
# ... the Hartree-Fock state for this system:
HF_state = QuantumState(data_dict['data']['hf_array'])
# ... and various classical quantum chemistry benchmark energies:
hf_energy = data_dict['data']['calculated_properties']['HF']['energy']
mp2_energy = data_dict['data']['calculated_properties']['MP2']['energy']
ccsd_energy = data_dict['data']['calculated_properties']['CCSD']['energy']
fci_energy = data_dict['data']['calculated_properties']['FCI']['energy']

/home/tweaving/qc-research/quantum-error-mitigation


# Tapering the Hamiltonian:

Remove $\mathbb{Z}_2$ symmetries, reducing the problem from $20 \rightarrow 16$ qubits.

In [55]:
from symmer import QubitTapering

QT = QubitTapering(H)

print(f'Qubit tapering permits a reduction of {H.n_qubits} -> {H.n_qubits-QT.n_taper} qubits.\n')
print('The following symmetry generators were identified:\n')
print(QT.symmetry_generators); print()
print('which we may rotate onto the single-qubit Pauli operators\n') 
print(QT.symmetry_generators.rotate_onto_single_qubit_paulis()); print()
print('via a sequence of Clifford operations R_k = e^{i pi/4 P_k} where:\n')
for index, (P_k, angle) in enumerate(QT.symmetry_generators.stabilizer_rotations):
    P_k.sigfig=0
    print(f'P_{index} = {P_k}')

Qubit tapering permits a reduction of 20 -> 16 qubits.

The following symmetry generators were identified:

 1 IIIIIIIIZZIIIIIIZZII 
 1 IIIIIIZZIIIIIIZZIIII 
 1 IZIZIZIZIZIZIZIZIZIZ 
 1 ZIZIZIIZIZZIZIIZIZZI

which we may rotate onto the single-qubit Pauli operators

-1 IIIIIIIIXIIIIIIIIIII 
-1 IIIIIIXIIIIIIIIIIIII 
-1 IXIIIIIIIIIIIIIIIIII 
-1 XIIIIIIIIIIIIIIIIIII

via a sequence of Clifford operations R_k = e^{i pi/4 P_k} where:

P_0 =  1+0j IIIIIIIIYZIIIIIIZZII
P_1 =  1+0j IIIIIIYZIIIIIIZZIIII
P_2 =  1+0j IYIZIZIZIZIZIZIZIZIZ
P_3 =  1+0j YIZIZIIZIZZIZIIZIZZI


In [56]:
# We may pater the Hamiltonian with respect to the Hartree-Fock state (this defines the symmetry sector)
H_taper = QT.taper_it(ref_state=HF_state)
# ... along with any auxiliary operators one might be interested in:
UCC_taper = QT.taper_it(aux_operator=UCC)
NUM_taper = QT.taper_it(aux_operator=NUM)
# Note the number operator still commutes with the Hamiltonian in the reduced subspace:
print(f'Tapered number operator commutes with Hamiltonian? {H_taper.commutes(NUM_taper)}')

Tapered number operator commutes with Hamiltonian? True


# Contextual Subspace

Initialize an instance of the `ContextualSubspace` class, which shall extract a noncontextual component from the Hamiltonian and solve it classically.

In [57]:
from symmer import ContextualSubspace

CS = ContextualSubspace(H_taper, noncontextual_strategy='diag', reference_state=QT.tapered_ref_state)
CS.noncontextual_operator.energy, hf_energy

/home/tweaving/anaconda3/envs/symmer/lib/python3.8/site-packages/symmer/symplectic/independent_op.py:102: UserWarning: Greedy method may identify non-optimal commuting symmetry terms; might be able to taper again.
  warnings.warn('Greedy method may identify non-optimal commuting symmetry terms; might be able to taper again.')


(-455.13544567081203, -455.1354456708121)

We identify a contextual subspace that reduces our problem from $16 \rightarrow 3$ qubits while retaining a ground state energy within chemical accuracy of the full-configuration interaction (FCI) energy

In [58]:
# Choose the number of qubits one wishes to project onto,
# specifying the UCC operator to motivate the subspace choice:
CS.update_stabilizers(n_qubits=3, aux_operator=UCC_taper)
# and finally project the Hamiltonian into the contextual subspace:
H_cs = CS.project_onto_subspace()
print(H_cs)

-453.091+0.000j III +
 0.394+0.000j IIZ +
 0.621+0.000j IZI +
 0.847+0.000j IZZ +
 0.621+0.000j ZII +
 0.847+0.000j ZIZ +
 0.258+0.000j ZZI +
 0.238+0.000j ZZZ +
-0.015+0.000j IIX +
 0.015-0.000j IZX +
 0.015+0.000j ZIX +
-0.015+0.000j ZZX +
 0.005+0.000j IXI +
-0.005+0.000j IXZ +
 0.062+0.000j ZXI +
-0.062+0.000j ZXZ +
-0.010+0.000j IXX +
-0.010+0.000j IYY +
 0.010+0.000j ZXX +
 0.010+0.000j ZYY +
-0.005+0.000j XII +
 0.005+0.000j XIZ +
-0.062+0.000j XZI +
 0.062+0.000j XZZ +
 0.010+0.000j XIX +
-0.010+0.000j XZX +
 0.010-0.000j YIY +
-0.010+0.000j YZY +
-0.056+0.000j YYI +
 0.056+0.000j YYZ +
-0.035+0.000j XXX +
-0.035+0.000j XYY +
-0.035+0.000j YXY +
 0.035-0.000j YYX


In [59]:
cs_nrg, cs_psi = exact_gs_energy(H_cs.to_sparse_matrix)
error = abs(cs_nrg - fci_energy)
print(f'Energy error w.r.t FCI: {error} Ha')
print(f'Within chemical precision?: {error < 0.0016}')

Energy error w.r.t FCI: 0.000837380287691758 Ha
Within chemical precision?: True
